In [32]:
DEBUG = True
model_name = 'mobilenetv3_large_100'

In [33]:
from PIL import Image
import torchvision.transforms as transforms
import torch

In [34]:
PATH = 'model/' + model_name + '.pth'
model = torch.load(PATH)

In [35]:
def recognize(image_path):
    """Print top 5 possiblilities of the given image.

    Args:
      image_path: the path of the image

    Returns:
    """
    img = Image.open(image_path).convert('RGB')
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((224,224)),
    ])
    tensor = transform(img).unsqueeze(0)
    
    model.eval()
    with torch.no_grad():
        out = model(tensor)
    probabilities = torch.nn.functional.softmax(out[0], dim=0)
    
    with open("imagenet_classes.txt", "r") as f:
        categories = [s.strip() for s in f.readlines()]
    top5_prob, top5_catid = torch.topk(probabilities, 5)
    for i in range(top5_prob.size(0)):
        print(categories[top5_catid[i]], top5_prob[i].item())

In [36]:
recognize('image/dog.jpg')

Samoyed 0.8151047229766846
Pomeranian 0.02057637646794319
white wolf 0.014584108255803585
Persian cat 0.011715124361217022
Angora 0.009728971868753433
